In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/24 13:47:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-23 13:10:15|  2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [15]:
rows = rdd.take(10)
row = rows[0]

In [18]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

# GroupBys - map, filter, reduceByKey

In [6]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [31]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2020, 2, 1, 1, 0), 7), (36.3, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 74), (4.3, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 41), (8.16, 1)),
 ((datetime.datetime(2020, 2, 1, 5, 0), 129), (7.3, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 7), (9.8, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 97), (6.3, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 82), (8.76, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 168), (11.8, 1)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 41), (10.3, 1)),
 ((datetime.datetime(2020, 2, 1, 4, 0), 7), (4.8, 1))]

In [7]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [38]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(5)

[((datetime.datetime(2020, 2, 1, 1, 0), 7), (251.62000000000003, 16)),
 ((datetime.datetime(2020, 2, 1, 5, 0), 129), (102.07, 4)),
 ((datetime.datetime(2020, 2, 1, 0, 0), 168), (11.8, 1)),
 ((datetime.datetime(2020, 2, 1, 23, 0), 256), (229.19000000000005, 11)),
 ((datetime.datetime(2020, 2, 13, 10, 0), 55), (148.48000000000002, 4))]

In [8]:
from collections import namedtuple

In [9]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [10]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [36]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .take(5)

[RevenueRow(hour=datetime.datetime(2020, 2, 1, 1, 0), zone=7, revenue=251.62000000000003, count=16),
 RevenueRow(hour=datetime.datetime(2020, 2, 1, 5, 0), zone=129, revenue=102.07, count=4),
 RevenueRow(hour=datetime.datetime(2020, 2, 1, 0, 0), zone=168, revenue=11.8, count=1),
 RevenueRow(hour=datetime.datetime(2020, 2, 1, 23, 0), zone=256, revenue=229.19000000000005, count=11),
 RevenueRow(hour=datetime.datetime(2020, 2, 13, 10, 0), zone=55, revenue=148.48000000000002, count=4)]

# From RDD to DataFrame

In [11]:
from pyspark.sql import types

In [12]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [13]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [50]:
df_result.write.parquet('tmp/green-revenue')

# mapPartitions - for ML

In [4]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [5]:
import pandas as pd

In [6]:
rows = duration_rdd.take(10)
rows

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 11, 7, 42), PULocationID=179, DOLocationID=223, trip_distance=0.76),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 8, 22, 29), PULocationID=41, DOLocationID=237, trip_distance=3.32),
 Row(VendorID=2, lpep_pickup_datetime=d

In [7]:
df = pd.DataFrame(rows, columns=columns)

In [8]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [9]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5 # as a placeholder for a proper ML model
    return y_pred

In [98]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [102]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [104]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|             12.95|
|             31.25|
|              14.0|
|             12.75|
|               0.1|
|             11.05|
|11.299999999999999|
|54.349999999999994|
|             15.25|
|             91.75|
|             12.25|
|               3.1|
|               7.5|
|11.899999999999999|
| 78.89999999999999|
|              4.45|
|              23.2|
|              4.85|
|              6.65|
|              15.1|
+------------------+
only showing top 20 rows

